In [2]:
import numpy as np
import pandas as pd
import sklearn.model_selection as ms
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from statsmodels.stats.weightstats import DescrStatsW
from sklearn.linear_model import LogisticRegression
import scipy

In [2]:
df = pd.read_csv('diamonds.txt', sep='\t')

In [3]:
y = df.price

In [4]:
X = df.drop(columns='price')

In [7]:
trainX, testX, trainY, testY = ms.train_test_split(X, y, test_size=0.25, random_state=1)

In [9]:
lr = LinearRegression()
lr.fit(trainX, trainY)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [10]:
rf = RandomForestRegressor(random_state=1)
rf.fit(trainX, trainY)

/Users/avvallack/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [11]:
predLR = lr.predict(testX)
predRF = rf.predict(testX)

In [13]:
maeLR = abs(predLR - testY)
maeRF = abs(predRF - testY)

In [15]:
stats.ttest_rel(maeLR, maeRF)

Ttest_relResult(statistic=13.01772978387856, pvalue=1.6551745751413995e-38)

In [19]:
DescrStatsW(maeLR - maeRF).tconfint_mean()

(74.28724532595444, 100.62452098634296)

In [20]:
Z = (9.57-9.5)/(0.4/np.sqrt(160))

In [22]:
2*(1 - stats.norm.cdf(abs(Z)))

0.026856695507523787

0.32410186177608225

In [28]:
df = pd.read_csv('banknotes.txt', sep='\t')
df.shape

(200, 7)

In [27]:
y = df.real.values
X1 = df[['X1', 'X2', 'X3']]
X2 = df[['X4', 'X5', 'X6']]

In [31]:
trainX1, testX1, trainX2, testX2, trainY, testY = ms.train_test_split(X1, X2, y, test_size=0.25, random_state=1)

In [37]:
lr1 = LogisticRegression()
lr2 = LogisticRegression()
lr1.fit(trainX1, trainY)
lr2.fit(trainX2, trainY)
pred1 = lr1.predict(testX1)
pred2 = lr2.predict(testX2)
difpred1 = abs(pred1 - testY)
difpred2 = abs(pred2 - testY)

/Users/avvallack/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/avvallack/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [34]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [35]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [38]:
proportions_diff_z_stat_rel(difpred1, difpred2)

2.9386041680175268

In [39]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [40]:
proportions_diff_z_stat_ind(difpred1, difpred2)

2.876412474452669

In [43]:
proportions_diff_confint_rel(difpred1, difpred2)

(0.059945206279614305, 0.3000547937203857)

In [49]:
Z = (541.4-525)/(100/np.sqrt(100))

In [50]:
(1 - stats.norm.cdf(Z))

0.05050258347410397

In [51]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [53]:
proportions_diff_z_test(proportions_diff_z_stat_rel(difpred1, difpred2))

0.0032969384555543435

In [55]:
n1 = 34
n2 = 16
p1 = 10/n1
p2 = 4/n2
P = (p1*n1 + p2*n2) / (n1 + n2)
proportions_diff_z_test((p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2)))

0.7458609174504707

In [58]:
from statsmodels.stats.descriptivestats import sign_test

In [84]:
arr = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])

In [61]:
sign_test(arr, 200)

(-2.0, 0.3437499999999999)

In [4]:
sample1 = [22,22,15,13,19,19,18,20,21,13,13,15]
sample2 = [17,18,18,15,12,4,14,15,10]


In [85]:
stats.wilcoxon(arr - 200)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

In [83]:
stats.wilcoxon(sample2, 200)

TypeError: len() of unsized object

In [6]:
stats.mannwhitneyu(sample1, sample2)

MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

In [70]:
df3 = pd.read_csv('challenger.txt', sep='\t')

In [71]:
df3.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [72]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [73]:
accident = df3[df3.Incident == 1].Temperature.values
nonaccident = df3[df3.Incident == 0].Temperature.values

In [81]:
np.random.seed(0)

ilec_median_scores = (list(map(np.mean, get_bootstrap_samples(accident, 1000))))
clec_median_scores = (list(map(np.mean, get_bootstrap_samples(nonaccident, 1000))))

print("95% confidence interval for the ILEC median repair time:",  stat_intervals(ilec_median_scores, 0.05))
print("95% confidence interval for the CLEC median repair time:",  stat_intervals(clec_median_scores, 0.05))
delta_median_scores = list(map(lambda x: x[1] - x[0], zip(ilec_median_scores, clec_median_scores)))
print("95% confidence interval for the difference between medians",  stat_intervals(delta_median_scores, 0.05))

95% confidence interval for the ILEC median repair time: [14.61428571 20.71607143]
95% confidence interval for the CLEC median repair time: [21.14359375 23.55046875]
95% confidence interval for the difference between medians [1.45040179 8.06457589]


In [79]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [80]:
np.random.seed(0)
permutation_test(accident, nonaccident, max_permutations = 10000)

0.0057